# 🧠 Vector Store e Embeddings: A Biblioteca da IA Moderna

**Módulo 7 de 15 - LangChain v0.2**

### Por Pedro Nunes Guth

---

Tá, mas o que é um Vector Store? Imagina que você tem uma biblioteca GIGANTE com milhões de livros, mas não tem um sistema de catalogação. Como você acharia o livro que precisa? Impossível, né?

É exatamente isso que fazemos com informações quando usamos Vector Stores! Transformamos textos em "impressões digitais matemáticas" (embeddings) e organizamos tudo de forma que a IA consiga achar rapidinho o que precisa.

Nos módulos anteriores, vimos como carregar e dividir documentos. Agora vamos aprender como fazer a IA "entender" e organizar essas informações para usar no RAG (que vem no próximo módulo)!

**🎯 O que vamos ver:**
- O que são embeddings na prática
- Como funciona um Vector Store
- Implementação com LangChain v0.2
- Similaridade semântica
- Preparação para RAG

## 🤔 Mas Afinal, O Que São Embeddings?

Tá, vou explicar com uma analogia que todo brasileiro entende: **DNA das palavras**!

Sabe como cada pessoa tem um DNA único que define suas características? Embeddings são como o "DNA semântico" das palavras e frases!

### 📊 Matematicamente Falando:

Um embedding é uma representação vetorial densa de texto em um espaço multidimensional (geralmente 768, 1024 ou 1536 dimensões).

$$\text{embedding}(\text{"cachorro"}) = [0.2, -0.1, 0.8, ..., 0.3] \in \mathbb{R}^n$$

onde $n$ é o número de dimensões do modelo.

### 🧪 A Mágica da Similaridade:

Palavras com significados similares têm embeddings próximos no espaço vetorial:

$$\text{similaridade}(v_1, v_2) = \frac{v_1 \cdot v_2}{||v_1|| \cdot ||v_2||}$$

Isso significa que "cachorro" e "cão" terão vetores bem próximos!

**💡 Dica do Pedro:** Pensa assim: se as palavras fossem pessoas numa festa, as que têm assuntos em comum ficariam conversando perto umas das outras!

In [ ]:
# Bora instalar as dependências!
!pip install langchain langchain-community langchain-google-genai
!pip install faiss-cpu sentence-transformers
!pip install numpy matplotlib seaborn plotly

In [ ]:
# Imports essenciais para nosso laboratório de embeddings
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

# LangChain imports - a nova geração!
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document

print("🚀 Todas as bibliotecas carregadas! Bora para a aventura dos vetores!")

In [ ]:
# Configurando nossa chave da Google (você vai precisar da sua!)
import os
from google.colab import userdata

# Se estiver no Colab, use:
# os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Se estiver local, descomente e coloque sua chave:
# os.environ["GOOGLE_API_KEY"] = "sua_chave_aqui"

print("🔐 Chaves configuradas! Vamos embeddar tudo!")

## 🎯 Primeiro Contato: Criando Nossos Primeiros Embeddings

Vamos começar com o básico: transformar texto em vetores! É como traduzir português para "matematiquês" que a IA entende.

### 🔧 Tipos de Modelos de Embedding:

1. **Google Embeddings**: Integrados com Gemini
2. **Sentence Transformers**: Gratuitos e poderosos
3. **OpenAI Embeddings**: Caros mas eficientes

**💡 Dica do Pedro:** Para estudar, use Sentence Transformers. Para produção séria, Google ou OpenAI!

In [ ]:
# Vamos criar nosso primeiro modelo de embedding!
# Usando um modelo gratuito para começar

# Modelo brasileiro que funciona bem em português!
embeddings_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={'device': 'cpu'}  # Usa CPU mesmo
)

print("🎉 Modelo de embedding carregado!")
print(f"📊 Dimensões do vetor: Vamos descobrir...")

In [ ]:
# Testando nosso modelo com frases brasileiras!
frases_teste = [
    "Eu amo programar em Python",
    "Adoro desenvolver com Python", 
    "Python é uma linguagem incrível",
    "Hoje está chovendo muito",
    "O clima está chuvoso",
    "Vou comer pizza hoje"
]

# Gerando embeddings para cada frase
print("🧠 Gerando embeddings...")
embeddings_frases = []

for i, frase in enumerate(frases_teste):
    embedding = embeddings_model.embed_query(frase)
    embeddings_frases.append(embedding)
    print(f"📝 Frase {i+1}: '{frase}'")
    print(f"🔢 Dimensões: {len(embedding)}")
    print(f"📊 Primeiros 5 valores: {embedding[:5]}")
    print("-" * 50)

print("✅ Todos os embeddings gerados!")

## 📈 Visualizando a Mágica dos Embeddings

Tá, mas como visualizar 384 dimensões? É impossível! Mas temos um truque: **PCA (Análise de Componentes Principais)**.

É como pegar uma foto 3D e fazer uma sombra 2D na parede - perdemos informação, mas conseguimos "ver" as relações!

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-07_img_01.png)

In [ ]:
# Vamos visualizar nossos embeddings em 2D!
# É como fazer uma "radiografia" dos vetores

# Convertendo para numpy array para facilitar
embeddings_array = np.array(embeddings_frases)
print(f"📊 Shape dos embeddings: {embeddings_array.shape}")

# Aplicando PCA para reduzir para 2D
pca = PCA(n_components=2, random_state=42)
embeddings_2d = pca.fit_transform(embeddings_array)

print(f"📉 Variância explicada: {pca.explained_variance_ratio_}")
print(f"📊 Total de variância capturada: {sum(pca.explained_variance_ratio_):.2%}")

In [ ]:
# Criando o gráfico dos embeddings - a visualização da mágica!
plt.figure(figsize=(12, 8))

# Cores diferentes para temas diferentes
cores = ['red', 'red', 'red', 'blue', 'blue', 'green']
temas = ['Python', 'Python', 'Python', 'Clima', 'Clima', 'Comida']

# Plotando os pontos
for i, (x, y) in enumerate(embeddings_2d):
    plt.scatter(x, y, c=cores[i], s=100, alpha=0.7)
    plt.annotate(f"{i+1}: {temas[i]}", 
                xy=(x, y), 
                xytext=(5, 5), 
                textcoords='offset points',
                fontsize=10,
                bbox=dict(boxstyle='round,pad=0.3', facecolor=cores[i], alpha=0.3))

plt.title('🎯 Visualização dos Embeddings - Frases Similares Ficam Juntas!', fontsize=14)
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.grid(True, alpha=0.3)

# Legenda
plt.scatter([], [], c='red', label='Programação Python', s=100)
plt.scatter([], [], c='blue', label='Clima/Chuva', s=100)
plt.scatter([], [], c='green', label='Comida', s=100)
plt.legend()

plt.tight_layout()
plt.show()

print("🎨 Liiindo! Viu como as frases similares ficam próximas?")

## 🔍 Calculando Similaridades: A Matemática do "Parecido"

Agora vamos calcular exatamente o quão "parecidas" são nossas frases usando **similaridade cosseno**.

### 📐 Fórmula da Similaridade Cosseno:

$$\cos(\theta) = \frac{A \cdot B}{||A|| \cdot ||B||} = \frac{\sum_{i=1}^{n} A_i B_i}{\sqrt{\sum_{i=1}^{n} A_i^2} \sqrt{\sum_{i=1}^{n} B_i^2}}$$

**Onde:**
- 1.0 = Idênticos
- 0.0 = Sem relação  
- -1.0 = Opostos

**💡 Dica do Pedro:** É como medir o ângulo entre dois vetores. Quanto menor o ângulo, mais parecidos!

In [ ]:
# Calculando a matriz de similaridade - quem é parecido com quem?
similaridade_matrix = cosine_similarity(embeddings_array)

print("🧮 Matriz de Similaridade Cosseno:")
print("(1.0 = idênticos, 0.0 = sem relação)\n")

# Criando labels mais legíveis
labels = [
    "Amo Python",
    "Adoro Python", 
    "Python incrível",
    "Chovendo muito",
    "Clima chuvoso",
    "Comer pizza"
]

# Mostrando a matriz bonitinha
for i, frase1 in enumerate(labels):
    for j, frase2 in enumerate(labels):
        if i <= j:  # Só mostra a parte superior da matriz
            score = similaridade_matrix[i][j]
            emoji = "🔥" if score > 0.8 and i != j else "✅" if score > 0.6 and i != j else "➡️" if i == j else "❌"
            print(f"{emoji} '{frase1}' vs '{frase2}': {score:.3f}")
    print()

In [ ]:
# Criando um heatmap da similaridade - visual e bonito!
plt.figure(figsize=(10, 8))

# Heatmap com seaborn
sns.heatmap(similaridade_matrix, 
            xticklabels=labels,
            yticklabels=labels,
            annot=True,
            fmt='.2f',
            cmap='RdYlBu_r',
            center=0.5,
            square=True,
            cbar_kws={'label': 'Similaridade Cosseno'})

plt.title('🔥 Mapa de Calor da Similaridade\nQuanto mais vermelho, mais parecido!', fontsize=14)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

print("🎨 Agora dá pra ver claramente quem é parecido com quem!")

## 🗄️ Vector Stores: A Biblioteca Organizada da IA

Tá, mas e se tivermos milhões de documentos? Não dá pra calcular similaridade com tudo na mão!

Entra o **Vector Store** - é como uma biblioteca super organizada onde:
- 📚 Cada livro (documento) tem sua "impressão digital" (embedding)
- 🔍 Quando você procura algo, ele acha os mais parecidos rapidinho
- ⚡ Usa algoritmos especiais (como FAISS) para busca ultra-rápida

### 🏗️ Como Funciona um Vector Store:

```mermaid
graph TD
    A[Documento Original] --> B[Gerar Embedding]
    B --> C[Armazenar no Vector Store]
    D[Query do Usuário] --> E[Gerar Embedding da Query]
    E --> F[Buscar Similares no Vector Store]
    F --> G[Retornar Documentos Mais Relevantes]
    C --> F
```

**💡 Dica do Pedro:** É como o Spotify que recomenda músicas parecidas com o que você gosta, mas para textos!

In [ ]:
# Vamos criar nosso primeiro Vector Store!
# Usando FAISS - Facebook AI Similarity Search (gratuito e rápido)

# Primeiro, criamos documentos estruturados (lembra do módulo 6?)
documentos_exemplo = [
    Document(
        page_content="Python é uma linguagem de programação interpretada de alto nível.",
        metadata={"fonte": "wikipedia", "categoria": "programação"}
    ),
    Document(
        page_content="LangChain é um framework para desenvolver aplicações com modelos de linguagem.",
        metadata={"fonte": "docs", "categoria": "programação"}
    ),
    Document(
        page_content="Inteligência artificial está revolucionando a tecnologia moderna.",
        metadata={"fonte": "artigo", "categoria": "tecnologia"}
    ),
    Document(
        page_content="Machine learning permite que computadores aprendam sem programação explícita.",
        metadata={"fonte": "livro", "categoria": "tecnologia"}
    ),
    Document(
        page_content="O futebol é o esporte mais popular do Brasil.",
        metadata={"fonte": "esporte", "categoria": "esporte"}
    ),
    Document(
        page_content="A culinária brasileira é muito diversificada e saborosa.",
        metadata={"fonte": "gastronomia", "categoria": "cultura"}
    )
]

print(f"📚 Criamos {len(documentos_exemplo)} documentos para nosso vector store!")
print("\n📋 Resumo dos documentos:")
for i, doc in enumerate(documentos_exemplo):
    print(f"{i+1}. {doc.page_content[:50]}... [{doc.metadata['categoria']}]")

In [ ]:
# Criando nosso Vector Store com FAISS
print("🏗️ Construindo o Vector Store...")
print("📊 Gerando embeddings para todos os documentos...")

# FAISS.from_documents faz toda a mágica:
# 1. Gera embeddings para cada documento
# 2. Cria o índice FAISS
# 3. Armazena tudo organizadinho
vector_store = FAISS.from_documents(
    documents=documentos_exemplo,
    embedding=embeddings_model
)

print("✅ Vector Store criado com sucesso!")
print(f"📈 Número de documentos indexados: {vector_store.index.ntotal}")
print("🚀 Agora podemos fazer buscas semânticas ultra-rápidas!")

## 🔍 Busca Semântica: A Mágica Acontece Aqui!

Agora vem a parte mais **LINDA** do Vector Store: busca semântica!

Diferente de busca tradicional (que procura palavras exatas), a busca semântica entende o **significado**!

### 🆚 Comparação:
- **Busca Tradicional:** "cachorro" ≠ "cão" 
- **Busca Semântica:** "cachorro" ≈ "cão" ≈ "pet" ≈ "animal doméstico"

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-07_img_02.png)

In [ ]:
# Vamos testar nossa busca semântica!
def busca_semantica(query, k=3):
    """Função para fazer busca semântica e mostrar resultados bonitinhos"""
    print(f"🔍 Buscando por: '{query}'")
    print("="*60)
    
    # Busca por similaridade
    resultados = vector_store.similarity_search_with_score(query, k=k)
    
    for i, (doc, score) in enumerate(resultados, 1):
        # Convertendo score do FAISS (distância) para similaridade
        similaridade = 1 / (1 + score)  # Quanto menor a distância, maior a similaridade
        
        print(f"🏆 Resultado {i} - Similaridade: {similaridade:.3f}")
        print(f"📝 Conteúdo: {doc.page_content}")
        print(f"🏷️ Categoria: {doc.metadata['categoria']}")
        print(f"📊 Score FAISS: {score:.3f}")
        print("-" * 40)
    
    return resultados

print("🎯 Função de busca semântica pronta! Vamos testar...")

In [ ]:
# Teste 1: Buscando sobre programação
resultados1 = busca_semantica("Como programar em linguagens de código?", k=3)

In [ ]:
# Teste 2: Buscando sobre IA/tecnologia
resultados2 = busca_semantica("Algoritmos que aprendem sozinhos", k=3)

In [ ]:
# Teste 3: Buscando sobre cultura brasileira
resultados3 = busca_semantica("Tradições e comida do Brasil", k=2)

## ⚙️ Configurações Avançadas do Vector Store

Bora turbinar nosso Vector Store! Tem várias configurações que fazem diferença na qualidade da busca.

### 🎛️ Parâmetros Importantes:

1. **k**: Quantos resultados retornar
2. **fetch_k**: Quantos candidatos buscar internamente (sempre >= k)
3. **lambda_mult**: Para MMR (Maximum Marginal Relevance) - diversidade vs relevância
4. **score_threshold**: Filtro por score mínimo

**💡 Dica do Pedro:** Fetch_k maior = busca mais precisa mas mais lenta. É o trade-off clássico!

In [ ]:
# Testando diferentes tipos de busca
query_teste = "Desenvolvimento de software com IA"

print("🧪 Comparando diferentes tipos de busca:\n")

# 1. Busca simples por similaridade
print("1️⃣ BUSCA POR SIMILARIDADE SIMPLES:")
docs_similaridade = vector_store.similarity_search(query_teste, k=2)
for i, doc in enumerate(docs_similaridade, 1):
    print(f"   {i}. {doc.page_content[:60]}...")

print("\n" + "="*60 + "\n")

# 2. Busca com score
print("2️⃣ BUSCA COM SCORE:")
docs_com_score = vector_store.similarity_search_with_score(query_teste, k=2)
for i, (doc, score) in enumerate(docs_com_score, 1):
    print(f"   {i}. Score: {score:.3f} - {doc.page_content[:50]}...")

print("\n" + "="*60 + "\n")

# 3. MMR (Maximum Marginal Relevance) - busca diversificada
print("3️⃣ BUSCA COM MMR (diversificada):")
docs_mmr = vector_store.max_marginal_relevance_search(
    query_teste, 
    k=2, 
    fetch_k=4,  # Busca 4 candidatos internamente
    lambda_mult=0.7  # 0.7 = balanceado, 1.0 = só relevância, 0.0 = só diversidade
)
for i, doc in enumerate(docs_mmr, 1):
    print(f"   {i}. {doc.page_content[:60]}...")

print("\n✅ Viu a diferença? MMR tenta trazer resultados relevantes MAS diversos!")

## 📊 Análise de Performance: Medindo a Qualidade

Como saber se nosso Vector Store tá funcionando bem? Vamos criar métricas!

### 📈 Métricas Importantes:

1. **Tempo de busca**: Velocidade das consultas
2. **Precisão**: Relevância dos resultados
3. **Diversidade**: Variedade dos resultados
4. **Cobertura**: Capacidade de achar informação relevante

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-07_img_03.png)

In [ ]:
# Vamos medir a performance do nosso Vector Store!
import time

def medir_performance_busca(queries, vector_store, k=3, num_testes=5):
    """Mede tempo e analisa qualidade das buscas"""
    resultados_performance = []
    
    for query in queries:
        print(f"⏱️ Testando query: '{query}'")
        tempos = []
        
        # Múltiplos testes para média confiável
        for i in range(num_testes):
            start_time = time.time()
            docs = vector_store.similarity_search(query, k=k)
            end_time = time.time()
            tempos.append(end_time - start_time)
        
        tempo_medio = np.mean(tempos)
        tempo_std = np.std(tempos)
        
        # Análise de diversidade (categorias diferentes nos resultados)
        categorias = [doc.metadata['categoria'] for doc in docs]
        diversidade = len(set(categorias)) / len(categorias)
        
        resultados_performance.append({
            'query': query,
            'tempo_medio': tempo_medio,
            'tempo_std': tempo_std,
            'diversidade': diversidade,
            'categorias_encontradas': categorias
        })
        
        print(f"   ⚡ Tempo médio: {tempo_medio*1000:.2f}ms (±{tempo_std*1000:.2f}ms)")
        print(f"   🎯 Diversidade: {diversidade:.2f}")
        print(f"   📋 Categorias: {categorias}")
        print()
    
    return resultados_performance

# Queries de teste
queries_teste = [
    "programação e desenvolvimento",
    "inteligência artificial",
    "cultura brasileira",
    "aprendizado de máquina"
]

print("🚀 Iniciando testes de performance...\n")
performance_results = medir_performance_busca(queries_teste, vector_store)

In [ ]:
# Visualizando os resultados de performance
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico 1: Tempo de busca
queries = [r['query'] for r in performance_results]
tempos = [r['tempo_medio']*1000 for r in performance_results]  # em ms
erros = [r['tempo_std']*1000 for r in performance_results]

bars1 = ax1.bar(range(len(queries)), tempos, yerr=erros, 
                capsize=5, alpha=0.7, color='skyblue')
ax1.set_title('⚡ Tempo de Busca por Query', fontsize=12)
ax1.set_ylabel('Tempo (ms)')
ax1.set_xticks(range(len(queries)))
ax1.set_xticklabels([q[:15] + '...' if len(q) > 15 else q for q in queries], 
                    rotation=45, ha='right')

# Adicionando valores nas barras
for i, bar in enumerate(bars1):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + erros[i]/2,
             f'{height:.1f}ms', ha='center', va='bottom', fontsize=9)

# Gráfico 2: Diversidade dos resultados
diversidades = [r['diversidade'] for r in performance_results]
bars2 = ax2.bar(range(len(queries)), diversidades, 
                alpha=0.7, color='lightcoral')
ax2.set_title('🎯 Diversidade dos Resultados', fontsize=12)
ax2.set_ylabel('Score de Diversidade (0-1)')
ax2.set_ylim(0, 1)
ax2.set_xticks(range(len(queries)))
ax2.set_xticklabels([q[:15] + '...' if len(q) > 15 else q for q in queries], 
                    rotation=45, ha='right')

# Adicionando valores nas barras
for i, bar in enumerate(bars2):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.02,
             f'{height:.2f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

# Estatísticas resumo
tempo_total_medio = np.mean([r['tempo_medio'] for r in performance_results])
diversidade_media = np.mean([r['diversidade'] for r in performance_results])

print(f"📊 RESUMO DA PERFORMANCE:")
print(f"⚡ Tempo médio de busca: {tempo_total_medio*1000:.2f}ms")
print(f"🎯 Diversidade média: {diversidade_media:.2f}")
print(f"💡 Vector Store funcionando {'🚀 EXCELENTE!' if tempo_total_medio < 0.01 else '✅ BEM!'}")

## 💾 Salvando e Carregando Vector Stores

E se quisermos salvar nosso Vector Store? Não queremos ficar recriando embeddings toda hora!

### 🔄 Persistence Strategies:

1. **Local**: Salva em disco (FAISS suporta nativamente)
2. **Cloud**: AWS S3, Google Cloud Storage
3. **Database**: PostgreSQL com pgvector, Pinecone, Weaviate

**💡 Dica do Pedro:** Para desenvolvimento, use local. Para produção, considere soluções cloud!

In [ ]:
# Salvando nosso Vector Store
import os

# Criando diretório se não existir
vector_store_path = "./meu_vector_store"
os.makedirs(vector_store_path, exist_ok=True)

print("💾 Salvando Vector Store...")

# FAISS permite salvar o índice
vector_store.save_local(vector_store_path)

print(f"✅ Vector Store salvo em: {vector_store_path}")
print("📁 Arquivos criados:")
for arquivo in os.listdir(vector_store_path):
    tamanho = os.path.getsize(os.path.join(vector_store_path, arquivo))
    print(f"   📄 {arquivo}: {tamanho} bytes")

In [ ]:
# Carregando Vector Store salvo
print("📂 Carregando Vector Store do disco...")

# Carregando o vector store salvo
vector_store_carregado = FAISS.load_local(
    vector_store_path, 
    embeddings_model,
    allow_dangerous_deserialization=True  # Necessário para arquivos locais
)

print("✅ Vector Store carregado com sucesso!")
print(f"📊 Documentos no índice: {vector_store_carregado.index.ntotal}")

# Testando se funciona
print("\n🧪 Testando vector store carregado:")
teste_docs = vector_store_carregado.similarity_search("Python programming", k=2)
for i, doc in enumerate(teste_docs, 1):
    print(f"   {i}. {doc.page_content[:50]}...")

print("\n🎉 Liiindo! Funciona perfeitamente!")

## 🏋️‍♀️ Exercício Prático 1: Construa Seu Próprio Vector Store

Agora é sua vez! Vamos criar um Vector Store com documentos sobre o Brasil.

**🎯 Seu Desafio:**
1. Criar 8 documentos sobre diferentes aspectos do Brasil
2. Construir um Vector Store
3. Fazer 3 buscas diferentes
4. Analisar os resultados

**💡 Dica do Pedro:** Pense em categorias como: geografia, cultura, economia, história, esporte, culinária!

In [ ]:
# SEU CÓDIGO AQUI!
# 1. Crie uma lista com 8 Document objects sobre o Brasil
# Exemplo de estrutura:

meus_documentos_brasil = [
    # Document(
    #     page_content="Sua informação sobre o Brasil aqui...",
    #     metadata={"categoria": "geografia", "fonte": "exemplo"}
    # ),
    # ... adicione mais 7 documentos
]

# SEU CÓDIGO PARA:
# 2. Criar o vector store
# 3. Fazer 3 buscas diferentes
# 4. Mostrar os resultados

print("🚧 Escreva seu código aqui!")
print("💪 Você consegue! Bora praticar!")

## 🔬 Exercício Prático 2: Comparando Modelos de Embedding

Diferentes modelos de embedding podem dar resultados diferentes! Vamos comparar.

**🎯 Seu Desafio:**
1. Usar dois modelos diferentes (ex: multilingual vs português específico)
2. Fazer a mesma busca com ambos
3. Comparar os resultados
4. Analisar qual funciona melhor

**💡 Dica do Pedro:** Teste com frases em português bem brasileiro para ver a diferença!

In [ ]:
# EXERCÍCIO: Compare dois modelos de embedding

# Modelo 1: Já temos o multilingual
modelo1 = embeddings_model

# Modelo 2: Tente outro modelo (descomente uma opção):
# modelo2 = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# modelo2 = HuggingFaceEmbeddings(model_name="neuralmind/bert-base-portuguese-cased")

# SEU CÓDIGO AQUI!
# 1. Crie dois vector stores com o mesmo conteúdo mas modelos diferentes
# 2. Faça a mesma busca em ambos
# 3. Compare os resultados

print("🧪 Implemente a comparação de modelos aqui!")
print("🔬 Ciência de dados na prática!")

## 🚀 Conectando com o RAG: Preparando o Terreno

Agora que dominamos Vector Stores, vamos ver como isso se conecta com RAG (Retrieval-Augmented Generation) que veremos no **próximo módulo**!

### 🔗 Como Vector Store + RAG Funcionam Juntos:

```mermaid
graph LR
    A[Pergunta do Usuário] --> B[Vector Store]
    B --> C[Documentos Relevantes]
    C --> D[LLM + Contexto]
    D --> E[Resposta Fundamentada]
    
    F[Base de Conhecimento] --> B
```

**💡 Dica do Pedro:** O Vector Store é o "cérebro" que lembra das informações. O LLM é a "boca" que fala baseado no que o cérebro lembrou!

In [ ]:
# Preview do RAG - uma pitadinha do que vem por aí!
def preview_rag(query, vector_store, top_k=2):
    """Mini demonstração de como seria um RAG simples"""
    print(f"🤖 PREVIEW RAG: '{query}'\n")
    
    # 1. Busca documentos relevantes (Retrieval)
    print("1️⃣ RETRIEVAL - Buscando documentos relevantes...")
    docs_relevantes = vector_store.similarity_search(query, k=top_k)
    
    contexto = ""
    for i, doc in enumerate(docs_relevantes, 1):
        contexto += f"Documento {i}: {doc.page_content}\n"
        print(f"   📄 Doc {i}: {doc.page_content[:60]}...")
    
    print(f"\n2️⃣ AUGMENTATION - Preparando contexto para o LLM...")
    
    # 2. Monta prompt com contexto (Augmentation)
    prompt_rag = f"""
    Contexto relevante:
    {contexto}
    
    Pergunta: {query}
    
    Responda baseado APENAS no contexto fornecido:
    """
    
    print("   ✅ Contexto preparado!")
    print("\n3️⃣ GENERATION - Aqui entraria o LLM para gerar a resposta...")
    print("   🔮 No próximo módulo implementaremos isso completo!")
    
    return prompt_rag, docs_relevantes

# Testando nosso preview
print("🎬 Demonstração de como será o RAG completo:\n")
prompt_exemplo, docs_exemplo = preview_rag("Como aprender programação?", vector_store)

print("\n" + "="*50)
print("🚀 No Módulo 8, vamos implementar RAG completo com LangChain!")

## 📚 Resumo: O Que Aprendemos Hoje

**Liiindo!** Que jornada incrível pelos Vector Stores e Embeddings! 🎉

### ✅ Conceitos Dominados:

1. **🧠 Embeddings**: "DNA semântico" das palavras
   - Vetores multidimensionais que capturam significado
   - Similaridade cosseno para medir "parecimento"

2. **🗄️ Vector Stores**: Biblioteca organizada da IA
   - FAISS para busca ultra-rápida
   - Busca semântica vs busca tradicional

3. **🔍 Tipos de Busca**:
   - Similarity Search: Por relevância
   - MMR: Balanceando relevância e diversidade
   - Com scores: Para análise quantitativa

4. **⚡ Performance**: Medindo qualidade e velocidade

5. **💾 Persistence**: Salvando e carregando para reutilizar

### 🔗 Conexões com o Curso:
- **Módulo 6 ➡️ 7**: Document Loading → Vector Storage
- **Módulo 7 ➡️ 8**: Vector Storage → RAG Implementation

### 🎯 Próximos Passos:
No **Módulo 8**, vamos juntar tudo:
- Vector Store (✅ dominado)
- LLMs (✅ do Módulo 2)
- Prompts (✅ do Módulo 3)
- = **RAG COMPLETO** 🚀

**💡 Dica Final do Pedro:** Vector Stores são a base de 80% das aplicações de IA moderna. Você acabou de dominar uma das tecnologias mais importantes do momento!

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/langchain---usando-versão-v0.2-modulo-07_img_04.png)

## 🎊 Parabéns! Você é Oficialmente um Vector Store Expert!

**🏆 Certificado Imaginário:** *"Pedro Guth certifica que você domina Vector Stores e está pronto para RAG!"*

### 🚀 Para Casa:
1. Experimente diferentes modelos de embedding
2. Teste com seus próprios documentos
3. Brinque com os parâmetros de busca
4. Se prepare para o RAG no próximo módulo!

### 📖 Leituras Extras (Opcional):
- Documentação oficial do FAISS
- Sentence Transformers documentation
- Artigos sobre embedding models em português

**Bora para o próximo módulo implementar RAG completo! 🚀**

---
*"A diferença entre busca tradicional e semântica é como a diferença entre procurar pela palavra exata no dicionário vs entender o que a pessoa realmente quer dizer."* - Pedro Guth

**#VectorStore #Embeddings #LangChain #RAG #IA**